# 0. 회사리스트 불러오기

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

In [2]:
corplist = pd.read_csv('corplist200.csv', encoding='cp949')
corplist

,company,code
0,삼성전자,5930
1,SK하이닉스,660
2,NAVER,35420
3,LG화학,51910
4,현대차,5380
...,...,...
147,일양약품,7570
148,JW중외제약,1060
149,SPC삼립,5610
150,삼양사,145990


In [3]:
stock_code = list(corplist['code'])

In [4]:
def convertcode(code):
    code = str(code)
    code = '0'*(6-len(code))+code
    return code

# 1. DB생성

In [5]:
def createdb(code):
    con = sqlite3.connect('./뉴스/news_{}.db'.format(code))
    type(con)
    sqlite3.Connection
    cursor = con.cursor()
    cursor.execute("CREATE TABLE news(idx integer, title text, date text, url text, key text unique, content text, primary key(idx autoincrement))")

In [6]:
for code in stock_code:
    dbname = convertcode(code)
    createdb(dbname)

# 2. 뉴스 3사 DB 병합

In [7]:
def concatdb(company):
    code = corplist[corplist['company']==company]['code'].values[0]
    code = convertcode(code)
    dbname = './뉴스/news_{}.db'.format(code)
    db1 = './뉴스/연합인포맥스/yinfo_{}.db'.format(company)
    db2 = './뉴스/이데일리/{}_ed_news.db'.format(company)
    db3 = './뉴스/한경/{}_hk_news.db'.format(company)
    with sqlite3.connect(db1) as conn:
        cur=conn.cursor()
        sql='select * from news'
        result1 = cur.execute(sql).fetchall()
    with sqlite3.connect(db2) as conn:
        cur=conn.cursor()
        sql='select * from news'
        result2 = cur.execute(sql).fetchall()
    with sqlite3.connect(db3) as conn:
        cur=conn.cursor()
        sql='select * from news'
        result3 = cur.execute(sql).fetchall()
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        for i in range(len(result1)):
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            cur.execute(sql, result1[i][1:])
        for i in range(len(result2)):
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            cur.execute(sql, result2[i][1:])
        for i in range(len(result3)):
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            cur.execute(sql, result3[i][1:])
        sql = 'select count(*) from news'
        result = cur.execute(sql).fetchall()
        print(len(result1),len(result2),len(result3),result)

In [8]:
for i in range(len(corplist)):
    company = corplist['company'][i]
    print(company)
    try:
        concatdb(company)
    except:
        print('에러:',company)

삼성전자
830 2841 6092 [(9763,)]
SK하이닉스
1017 916 1518 [(3451,)]
NAVER
에러: NAVER
LG화학
243 593 1230 [(2066,)]
현대차
731 2998 5986 [(9715,)]
삼성바이오로직스
50 283 430 [(763,)]
카카오
564 2024 3106 [(5694,)]
삼성SDI
66 294 573 [(933,)]
기아차
382 1132 2148 [(3662,)]
POSCO
에러: POSCO
현대모비스
121 364 847 [(1332,)]
LG전자
452 1342 2793 [(4587,)]
LG생활건강
43 165 290 [(498,)]
삼성물산
135 356 597 [(1088,)]
KB금융
217 365 870 [(1452,)]
SK텔레콤
156 579 780 [(1515,)]
엔씨소프트
59 321 413 [(793,)]
SK이노베이션
71 0 465 [(536,)]
신한지주
31 96 164 [(291,)]
SK
에러: SK
삼성생명
165 200 627 [(992,)]
LG
에러: LG
한국전력
9 200 382 [(591,)]
삼성에스디에스
에러: 삼성에스디에스
아모레퍼시픽
86 279 519 [(884,)]
삼성전기
54 340 628 [(1022,)]
하나금융지주
9 62 134 [(205,)]
KT&G
에러: KT&G
롯데케미칼
70 199 259 [(528,)]
한온시스템
13 58 108 [(179,)]
S-Oil
에러: S-Oil
삼성화재
127 198 492 [(817,)]
LG디스플레이
89 311 486 [(886,)]
고려아연
1 44 97 [(142,)]
금호석유
12 81 154 [(247,)]
현대글로비스
57 181 329 [(567,)]
KT
에러: KT
기업은행
125 407 1623 [(2155,)]
CJ제일제당
98 395 460 [(953,)]
현대제철
83 228 363 [(674,)]
강원랜드
3 193 472 [(668,)]
LG유플러스
97

# 4. 병합에러 확인
- 병합에러.xlsx 참고
- 회사명과 파일명 불일치(ex.naver 네이버)

In [11]:
def concatdb2(company,code):
    dbname = './뉴스/news_{}.db'.format(code)
    db1 = './뉴스/연합인포맥스/yinfo_{}.db'.format(company)
    db2 = './뉴스/이데일리/{}_ed_news.db'.format(company)
    db3 = './뉴스/한경/{}_hk_news.db'.format(company)
    with sqlite3.connect(db1) as conn:
        cur=conn.cursor()
        sql='select * from news'
        result1 = cur.execute(sql).fetchall()
    with sqlite3.connect(db2) as conn:
        cur=conn.cursor()
        sql='select * from news'
        result2 = cur.execute(sql).fetchall()
    with sqlite3.connect(db3) as conn:
        cur=conn.cursor()
        sql='select * from news'
        result3 = cur.execute(sql).fetchall()
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        for i in range(len(result1)):
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            cur.execute(sql, result1[i][1:])
        for i in range(len(result2)):
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            cur.execute(sql, result2[i][1:])
        for i in range(len(result3)):
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            cur.execute(sql, result3[i][1:])
        sql = 'select count(*) from news'
        result = cur.execute(sql).fetchall()
        print(len(result1),len(result2),len(result3),result)

In [49]:
company = '대상'
code = '001680'
try:
    concatdb2(company,code)
except:
    print('에러:',company)

581 855 112 [(1548,)]


In [47]:
corplist[corplist['company']=='대상']

,company,code
138,대상,1680
